In [ ]:
import numpy as np
import jax.numpy as jnp
from jax.scipy.special import logsumexp
from numpy.random.mtrand import noncentral_chisquare
from torch._C import NoneType
import jax
import _pickle as cPickle
import pickle
import copy
from jax import jit, vmap, pmap, grad, value_and_grad
import random
from torchvision.datasets import MNIST
from torchvision.datasets import KMNIST
from torchvision.datasets import CIFAR100
from torchvision.datasets import EMNIST
from torchvision.datasets import FashionMNIST
from torch.utils.data import DataLoader
from jax.example_libraries import stax, optimizers
import torchvision
import torch
from numpy.random import default_rng
from sklearn.neighbors import NearestNeighbors
import torch.utils.data as data_utils
from jax.flatten_util import ravel_pytree
import os
import time
import shutil
import _pickle as cPickle
from math import e
import time
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense, Relu, LogSoftmax
from sklearn.model_selection import train_test_split
from jax import random 
import matplotlib.pyplot as plt
from torchvision import transforms
from logging import Logger
import time
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
known_npy = ['anvil',
 'bathtub',
 'bicycle',
 'baseball bat',
 'book',
 'arm',
 'asparagus',
 'airplane',
 'blackberry',
 'bee',
 'blueberry',
 'barn',
 'apple',
 'banana',
 'bench',
 'bear',
 'bandage',
 'The Great Wall of China',
 'basketball',
 'basket']

unknown_npy = ['beach',
 'aircraft carrier',
 'boomerang',
 'axe',
 'beard',
 'bat',
 'bird',
 'ambulance',
 'animal migration',
 'The Mona Lisa',
 'binoculars',
 'bed',
 'birthday cake',
 'angel',
 'alarm clock',
 'belt',
 'baseball',
 'The Eiffel Tower',
 'ant',
 'backpack']


if not any(".npy" in item for item in known_npy):
  known_npy = [x+".npy" for x in known_npy]

if not any(".npy" in item for item in unknown_npy):
  unknown_npy = [x+".npy" for x in unknown_npy]


In [ ]:
'''Load data from npy files and normalize and standardize
train_npy and test_npy set manually'''



data_image_size = 28

def normalize_standardize(arr):
  arr = np.array(arr)/255
  arr = (arr-arr.mean())/arr.std()
  return arr


x_train_known_dataset = []
x_test_known_dataset = []
y_train_known_dataset = []
y_test_known_dataset = []

x_train_unknown_dataset = []
x_test_unknown_dataset = []
y_train_unknown_dataset = []
y_test_unknown_dataset = []

known_list = []
unknown_list = []

data_image_size = 28
data_image_depth = 1
number_datapoints_npy = 5000
data_filepath = "./Quickdraw_data/"
#data_filepath = "/content/drive/MyDrive/Colab Notebooks/Handdraw/data/"



if not any(".npy" in item for item in known_npy):
  known_npy = [x+".npy" for x in known_npy]

for idx, files in enumerate(os.listdir(data_filepath)):
  if "npy" in files:
    file_path = data_filepath+files
    npy_load = np.load(file_path, encoding = 'latin1', allow_pickle = True)
    rng = default_rng(idx)
    idx_list = rng.choice(len(npy_load), size = number_datapoints_npy*2, replace = False)
    idx_list_train = idx_list[number_datapoints_npy:]
    idx_list_test = idx_list[:number_datapoints_npy]


    if files in known_npy:

      x_train_known_dataset.extend(npy_load[idx_list_train])
      x_test_known_dataset.extend(npy_load[idx_list_test])
      y_train_known_dataset.extend([idx]*number_datapoints_npy)
      y_test_known_dataset.extend([idx]*number_datapoints_npy)
      known_list.append(idx)

    if files in unknown_npy:
      x_train_unknown_dataset.extend(npy_load[idx_list_train])
      x_test_unknown_dataset.extend(npy_load[idx_list_test])
      y_train_unknown_dataset.extend([idx]*number_datapoints_npy)
      y_test_unknown_dataset.extend([idx]*number_datapoints_npy)
      unknown_list.append(idx)

x_train_known_dataset = normalize_standardize(x_train_known_dataset).reshape(-1,data_image_size,data_image_size)
y_train_known_dataset = np.array(y_train_known_dataset)
x_test_known_dataset = normalize_standardize(x_test_known_dataset).reshape(-1,data_image_size,data_image_size)
y_test_known_dataset = np.array(y_test_known_dataset)


y_train_unknown_dataset = np.array(y_train_unknown_dataset)
x_train_unknown_dataset = normalize_standardize(x_train_unknown_dataset).reshape(-1,data_image_size,data_image_size)
y_test_unknown_dataset = np.array(y_test_unknown_dataset)
x_test_unknown_dataset = normalize_standardize(x_test_unknown_dataset).reshape(-1,data_image_size,data_image_size)



## **Funktions**

Changes in Jax Jit function (sometimes?) need a cache restart for changes to be applied. Happened with def accuracy

In [ ]:
def pathandstuff():

    global save_txt
    global base_path
    global save_path

    if os.path.exists(googledrive_path):
        print("on google")
        base_path = googledrive_path
    else:
        raise ValueError('Please create folder for save path or connect to Google Drive')
        
    logs_path = base_path
    '''Set logging and temp paths'''
    timestamp = time.strftime("%d.%m.%Y_%H.%M")
    foldername = timestamp
    save_path = os.path.join(logs_path,foldername,)
    save_path = save_path+"/"
    save_txt = os.path.join(save_path, 'Log_Jax_MNist_{}.txt'.format(foldername))
      
    if use_pickle:
      save_path = pickle_path.split("best_weight_")[0] 
      for i in os.listdir(save_path):
        if "Log" in i:
          save_txt = os.path.join(save_path, i)

    print("Log path:",save_path)
    if not os.path.exists(save_path):
        os.makedirs(save_path)


'''Save current notebook'''
def logg_script(file_name, save_path):
  source = f"/content/drive/MyDrive/Colab Notebooks/{file_name}"
  destination = save_path+f"{file_name}.ipynb"
  shutil.copy2(source, destination)

'''logging to txt and print'''
def logg(string_, array = None):
  if array is None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write("\n")
    file1.close()
    print(string_)
  if array is not None:

    file1 = open(save_txt,"a")
    file1.write(string_)
    file1.write(str(array))
    file1.write("\n")
    file1.close()
    print(string_, array)

def log_variables():
    try:
    
      logg(("n_training_epochs = {}".format(n_training_epochs)))
      logg(("n_offsp_epoch = {}".format(n_offsp_epoch)))
      logg(("n_samples = {}".format(n_samples)))
      logg(("n_test = {}".format(n_test)))
      logg(("batch_size = {}".format(batch_size)))
      logg(("std_modifier = {}".format(std_modifier)))
      logg(("use_sigma_decay = {}".format(use_sigma_decay)))
      logg(("sigma_start = {}".format(sigma_start)))
      logg(("sigma_goal = {}".format(sigma_goal)))
      logg(("n_decay_epochs = {}".format(n_decay_epochs)))
      logg(("use_pickle = {}".format(use_pickle)))
      logg(("pickle_path = {}".format(pickle_path)))
      logg(("NNin1 = {}".format(NNin1)))
      logg(("NNout1 = {}".format(NNout1)))
      logg(("Convu_in1 = {}".format(Convu1_in)))
      logg(("Convu2_in = {}".format(Convu2_in)))
      logg(("Convu3_in = {}".format(Convu3_in)))
      logg(("kernelsize_ = {}".format(kernelsize_)))
      logg(("n_metaepochs = {}".format(n_metaepochs)))   
      logg(("n_offsp_epoch = {}".format(n_offsp_epoch)))
      logg(("n_offsprings = {}".format(n_offsprings)))
      logg(("use_softmax = {}".format(use_softmax)))
      logg(("temperature = {}".format(temperature)))
    except Exception:
      pass

In [ ]:
def logg_GPU():
  try:
    affe=!nvidia-smi
    string=""
    for i in affe:
      string+=i
    logg("GPU:",string.split("=========||   0  ")[1].split("            Off  | 000")[0])
  except:
    pass

In [ ]:
'''creating offsprings, filling treeleaf of 0 and 1 with gaussian noise, doesnt seem to be a problem, ex in offspring_list[0][5]'''


def jax_create_offsprings(key,n_offspr,  fath_weights,std_var):
  statedic_list = []
  randomizer_list = []
  for i in range(0,n_offspr):
    std_modifier = std_var
    rng = jax.random.PRNGKey(key+i)
    if np.random.rand()<threshold_randomizer and use_std_randomizer:
      std_modifier = std_var*factor_randomizer
      randomizer_list.append(i)
    random_value_tree = tree_random_normal_like(rng,fath_weights,std_modifier)
    son = jax.tree_map(lambda x,y: x+y, fath_weights,random_value_tree)
    statedic_list.append(son)
  if use_std_randomizer:
    return statedic_list, randomizer_list
  else:
    return statedic_list, []

def random_split_like_tree(rng_key, target = None, treedef = None):
    if treedef is None:
        treedef = jax.tree_util.tree_structure(target)
    keys = jax.random.split(rng_key, treedef.num_leaves)
    return jax.tree_util.tree_unflatten(treedef, keys)


def tree_random_normal_like(rng_key, target,std_modifier):
    keys_tree = random_split_like_tree(rng_key, target)
    return jax.tree_map(
        lambda l, k: jax.random.normal(k, l.shape, l.dtype)*std_modifier,
        target,
        keys_tree,
    )

In [ ]:
'''softmax for offspring list
    checked 11.04 working correctly'''
def softmax_offlist(off_list,acc_list,temp):
  softmax_list = softmax_result(acc_list,temp)
  for i in range(len(off_list)):
    if i == 0:
      top_dog = jax.tree_map(lambda x: x*softmax_list[i], off_list[i])
    else:
      general_dog = jax.tree_map(lambda x: x*softmax_list[i], off_list[i])
      top_dog = jax.tree_map(lambda x,y: x+y, top_dog,general_dog)
  return top_dog


'''Creates softmax/temp list out of accuracy list [0.2,0.3,....,0.8]'''
def softmax_result(results,temp: float):
    x = [z/temp for z in results]
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
def sigma_decay(start, end, n_iter):
  return(end/start)**(1/n_iter)

In [ ]:
def cat_dataloader(x,y,random_categories, random_state):
  x_train = np.empty((n_samples*number_different_catruns,data_image_size,data_image_size))
  x_test = np.empty((n_test*number_different_catruns,data_image_size,data_image_size))
  y_train = np.empty((n_samples*number_different_catruns,))
  y_test = np.empty((n_test*number_different_catruns,))

  for i in range(number_different_catruns):

    '''Choose 5 random categories from data'''
    x_data0 = x[np.isin(y, random_categories[i]).flatten()]
    y_data0 = y[np.isin(y, random_categories[i]).flatten()]

    '''relabel categories to 0-5 to avoid accuracy bug in mlp'''
    map_dic = dict(zip(random_categories[i], [0,1,2,3,4]))
    y_data0 = np.vectorize(map_dic.get)(y_data0)

    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(x_data0, y_data0, train_size = n_samples,
                                                  test_size = n_test,stratify = y_data0,
                                                  random_state = random_state)

    x_train[i*n_samples:(i+1)*n_samples] = x_train_temp
    x_test[i*n_test:(i+1)*n_test] = x_test_temp
    y_train[i*n_samples:(i+1)*n_samples] = y_train_temp
    y_test[i*n_test:(i+1)*n_test] = y_test_temp

  '''Cast to Jax Array'''
  x_train = jnp.array(x_train,dtype = "float32").reshape(len(x_train), -1)
  x_test = jnp.array(x_test,dtype = "float32").reshape(len(x_test), -1)
  y_train = jnp.array(y_train)
  y_test = jnp.array(y_test)

  '''Reshape for Jax Vectorization'''
  x_train = x_train.reshape(-1,int((n_samples/batch_size)),batch_size,data_image_size,data_image_size)
  y_train = y_train.reshape(-1,int((n_samples/batch_size)),batch_size)
  x_test = x_test.reshape(-1,n_test,data_image_size,data_image_size)
  y_test = y_test.reshape(-1,n_test)

  return x_train, x_test, y_train, y_test

In [ ]:
def init_MLP(layer_widths, parent_key, scale = 0.01):

    params = []
    keys = jax.random.split(parent_key, num = len(layer_widths)-1)

    for in_width, out_width, key in zip(layer_widths[:-1], layer_widths[1:], keys):
        weight_key, bias_key = jax.random.split(key)
        params.append([scale*jax.random.normal(weight_key, shape = (out_width, in_width)),
                       scale*jax.random.normal(bias_key, shape = (out_width,))
                       ])
    return params



In [ ]:
@jit
def MLP_predict(params, x):

    hidden_layers = params[:-1]
    activation = x

    for w, b in hidden_layers:
        activation = jax.nn.relu(jnp.dot(w, activation) + b)

    w_last, b_last = params[-1]
    logits = jnp.dot(w_last, activation) + b_last

    return logits - logsumexp(logits)

jit_MLP_predict = jit(MLP_predict)

@jit
def batched_MLP_predict(params,x):
  return vmap(jit_MLP_predict, (None, 0))(params,x)
  
jit_batched_MLP_predict = jit(batched_MLP_predict)


In [ ]:
Convu1_in = 32
Convu2_in = 64
kernelsize_ = (3,3)

conv_init, conv_apply = stax.serial(
    stax.Conv(Convu1_in,kernelsize_, padding = "SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Conv(Convu2_in, kernelsize_, padding = "SAME"),
    stax.BatchNorm(),
    stax.Relu,
    stax.MaxPool((2,2)),
    stax.Flatten,
    stax.Dense(50),
)


In [ ]:
def random_cat_list(categories, number_different_catruns, number_training_categories):
    combs = np.array(list(itertools.combinations(categories, number_training_categories)))
    sample = np.random.randint(len(combs), size = number_different_catruns)
    affe = np.array(combs[sample])
    return affe

In [ ]:
def load_best_weight_queue(pickle_path):
  count = 0
  best_weight_queue = []
  base_bath = pickle_path.split("best_weight_")[0]
  for idx,i in enumerate(os.listdir(base_bath)[::-1]):
    if "best_weight" in i and count < len_best_weight_queue:
      
      with open(base_bath+i, "rb") as input_file:
        best_weight_queue.append(cPickle.load(input_file))
      count = count+1
  print(f"Top {count} weights imported from Drive")
  return best_weight_queue

In [ ]:
@jit
def loss_fn(params, imgs, gt_lbls):
  
    predictions = jit_batched_MLP_predict(params, imgs)
    #print("predictions",predictions.shape)
    return -jnp.mean(predictions * gt_lbls)
    
jit_loss_fn = jit(loss_fn)


@jit
def update(step, params, imgs, gt_lbls, opt_state,lr = 0.01):
    loss, grads = value_and_grad(loss_fn)(params, imgs, gt_lbls)
    opt_state = opt_update(step, grads, opt_state)

    return loss, jax.tree_map(lambda p, g: p - lr*g, params, grads), opt_state

'''@jit
def update(params, imgs, gt_lbls,lr = 0.01):
    loss, grads = value_and_grad(loss_fn)(params, imgs, gt_lbls)

    return loss, jax.tree_map(lambda p, g: p - lr*g, params, grads)'''

jit_update = jit(update)

@jit
def accuracy(conv_weights,MLP_params, dataset_imgs, dataset_lbls):

    imgs = conv_apply(conv_weights, dataset_imgs.reshape(-1,data_image_size,data_image_size,data_image_depth))
    pred_classes = jnp.argmax(jit_batched_MLP_predict(MLP_params, imgs.reshape(-1,NNin1)), axis = 1)

    return jnp.mean(dataset_lbls ==  pred_classes+jnp.min(dataset_lbls))
    
jit_accuracy = jit(accuracy)


In [ ]:
'''For loop is neccesary to do batch training. Every update iteration needs to run with updated MPL params'''
@jit
def train(conv_weights, imgs, lbls,MLP_params,opt_state ):



  for n in range(n_training_epochs):  
    for i in range(jnp.shape(imgs)[0]):

      gt_labels = jax.nn.one_hot(lbls[i], number_training_categories)
      img_conv = conv_apply(conv_weights, imgs[i].reshape(-1,data_image_size,data_image_size,data_image_depth))
      #loss, MLP_params = jit_update(MLP_params, img_conv.reshape(-1,NNin1), gt_labels)
      step = (n+i)
      loss, MLP_params,opt_state = jit_update(step, MLP_params, img_conv.reshape(-1,NNin1), gt_labels,opt_state)
  return MLP_params
  
jit_train = jit(train)  

In [ ]:
@jit
def bootstrapp_offspring_MLP(params,opt_state,conv_weights, batch_affe, labelaffe,test_images,test_lbls):
  

  MLP_params_trained = jit_train(conv_weights, batch_affe, labelaffe,
                               #MLP_params,
                               params,opt_state)
 
  
  result = jit_accuracy(conv_weights,MLP_params_trained,test_images,test_lbls)
  return result,MLP_params_trained

jit_bootstrapp_offspring_MLP = jit(bootstrapp_offspring_MLP)  

@jit
def vmap_bootstrapp_offspring_MLP(params,opt_state,conv_weights, batch_affe, labelaffe,test_images,test_lbls):
  return vmap(jit_bootstrapp_offspring_MLP, ( None, None,None, 0,0,0,0))(params,opt_state,conv_weights, batch_affe, labelaffe,test_images,test_lbls)
  
jit_vmap_bootstrapp_offspring_MLP = jit(vmap_bootstrapp_offspring_MLP)


In [ ]:
'''Initialize Variables'''

number_training_categories = 5
NNin1 = 50
NNout1 = number_training_categories


googledrive_path = "/content/drive/MyDrive/Colab Notebooks/"
file_name = "Jax_Quickdraw_Experiment_2"

n_metaepochs = 5000 #Number of generations
n_offsprings = 50
n_training_epochs = 10 #gradient descent iterations with same data for training the second network
n_samples = 25 #n of training independent training samples for 2nd network - MLP, samples are stratified
batch_size = 5
number_different_catruns = 50 #number of second networks with different categories to take average performance of. 
n_offsp_epoch = 5 #repeat experiment with same offspring on different category sets

n_test = 75
test_every_x_meta = 10 #how often test for generalization


'''keys'''
starting_key = 260 #define starting point
MLP_start_key = 3610 #seed 
numpy_seed = 8450 #in create offsprings

use_n_elitist = False #weight update method
n_elitist = 10

use_Softmax = True #weight update method
use_father = True
std_modifier = 0.01
temp = 0.01

use_elitist = False #weight update method

use_sigma_decay = True #otherwise using constant sigma from config tab, decreasing sigma for random noise over time
n_decay_epochs = 500   # over how many metaepochs sigma is decayed
sigma_start = 0.05
sigma_goal = 0.01 #sigma goal after n_metaepochs


use_temp_decay = True
n_decay_epochs_temp = 500
temp_start = 0.05
temp_goal = 0.01

use_pickle = False #load weights
pickle_path = "./pretrained_params/best_weight_0.8463.pkl"
use_best_weights = True

use_best_weight_queue = True #keep track of the n best weights and use in every metaepoch
len_best_weight_queue = 10 


use_best_father_queue = True
len_best_father_queue = 10

use_std_randomizer = True
threshold_randomizer = 0.1
factor_randomizer = 10


# Main code

In [ ]:
'''Start Logging'''
pathandstuff()
logg_script(file_name, save_path)
log_variables()
logg_GPU()


'''Initialize variables'''
rng_MLP = jax.random.PRNGKey(MLP_start_key)
results_for_every_offspring = []
best_performer = 0
father_key = jax.random.PRNGKey(starting_key)
best_weights = conv_init(father_key, (batch_size,data_image_size,data_image_size,data_image_depth))[1]
common_start_acc = 0
best_weight_queue = []
best_father_queue = []
best_father_mean = 0
randomizer_list = []

MLP_params = init_MLP([NNin1, NNout1], rng_MLP)

# Defining an optimizer in Jax
step_size = 1e-3
opt_init, opt_update, get_params = optimizers.adam(step_size)

for meta in range (n_metaepochs):
    start_meta = time.time()

    '''Create category pairs'''
    random_training_categories = random_cat_list(known_list, number_different_catruns, number_training_categories)
    random_testing_categories = random_cat_list(unknown_list, number_different_catruns, number_training_categories)

    
    if use_sigma_decay:
      sigma_base = sigma_decay(sigma_start, sigma_goal, n_decay_epochs)
      if meta < n_decay_epochs:
        std_modifier = sigma_start*sigma_base**meta
      else:
        std_modifier = sigma_start*sigma_base**n_decay_epochs


    if use_temp_decay:
      temp_base = sigma_decay(temp_start, temp_goal, n_decay_epochs_temp)
      if meta < n_decay_epochs_temp:
        temp = temp_start*temp_base**meta
      else:
        temp = temp_start*temp_base**n_decay_epochs_temp

    '''Starting point'''
    if meta == 0:

      if use_pickle:
        with open(pickle_path, "rb") as input_file:
          father_weights = cPickle.load(input_file)
          print("pickle weights imported") 
        offspring_list,randomizer_list = jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
        if use_father:
          offspring_list[0] = father_weights

        if use_best_weight_queue:
          best_weight_queue = load_best_weight_queue(pickle_path)
          offspring_list.extend(best_weight_queue)
        
        save_path = pickle_path.split("best_weight_")[0]
        

      else:

        father_weights = conv_init(father_key, (batch_size,data_image_size,data_image_size,data_image_depth))
        father_weights = father_weights[1] ## Weights are actually stored in second element of two value tuple
        offspring_list,randomizer_list = jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
        if use_father:
          offspring_list[0] = father_weights


    '''Weight updates'''      
    if meta >= 1:

        '''Softmax Update'''
        
        if use_Softmax and not ((meta-1) % (test_every_x_meta) ==  0): #Disable weight update for testing epoch
          #print("using softmax")
          
          father_weights = softmax_offlist(offspring_list,results_for_every_offspring,temp)
          offspring_list,randomizer_list = jax_create_offsprings((meta+numpy_seed),n_offsprings, father_weights,std_modifier)
          if use_father:
            offspring_list[0] = father_weights
            offspring_list[1] = best_weights
            
        if use_elitist and not ((meta-1) % (test_every_x_meta) ==  0):
          
          best_offspring = offspring_list[(np.argmax(results_for_every_offspring))]
          offspring_list, randomizer_list = jax_create_offsprings((meta+numpy_seed),n_offsprings, best_offspring,std_modifier)
          if use_father:
              offspring_list[0] = father_weights
              offspring_list[1] = best_weights
              
        if use_n_elitist and not ((meta-1) % (test_every_x_meta) ==  0):
                      
          offspring_list_help = []

          indices = (-results_for_every_offspring).argsort()[:n_elitist]
          for idx in indices:
            help,helper_randomizer_list = jax_create_offsprings((meta+numpy_seed),int(n_offsprings/n_elitist), offspring_list[idx],std_modifier)
            randomizer_list.extend(helper_randomizer_list*n_elitist*idx)
            offspring_list_help.extend(help)
          if use_father:

              offspring_list[1] = best_weights
          offspring_list = offspring_list_help  
          
        if use_best_weight_queue:
           offspring_list.extend(best_weight_queue)     

        if use_best_father_queue:
          offspring_list.extend(best_father_queue)  
            
    results_overall = []
    #print("Overhead meta time:", time.time() - start_overhead)
    for oepoch in range(n_offsp_epoch):
      rng_MLP = jax.random.PRNGKey(MLP_start_key+meta+oepoch)  
      MLP_params = init_MLP([NNin1, NNout1], rng_MLP)
      start_data_collection_time = time.time()
      result_list_metaepoch = []
      '''Load "known data" for test_every_x_meta episodes'''
      if (meta % test_every_x_meta !=  0 or meta ==  0):
        if oepoch ==  0:
          logg(f"\tload known data for metaepoch:{meta}")
        random_state = (starting_key+meta+oepoch)

        x_train, _, y_train, _ = cat_dataloader(x_train_known_dataset,y_train_known_dataset,random_training_categories, random_state)
        _, x_test, _, y_test = cat_dataloader(x_test_known_dataset,y_test_known_dataset,random_training_categories, random_state)




        '''Load "unknown data" every test_every_x_meta run'''
      else:
        if oepoch ==  0:
          logg(f"\tload unknown data for metaepoch:{meta}")
        random_state = (starting_key+meta+oepoch)
        x_train, _, y_train, _ = cat_dataloader(x_train_unknown_dataset,y_train_unknown_dataset,random_testing_categories, random_state)
        _, x_test, _, y_test = cat_dataloader(x_test_unknown_dataset,y_test_unknown_dataset,random_testing_categories, random_state)

        '''Cast to Jax Array'''
      x_train = jnp.array(x_train,dtype = "float32").reshape(number_different_catruns,int((n_samples/batch_size)),batch_size,data_image_size,data_image_size)
      x_test = jnp.array(x_test,dtype = "float32").reshape(number_different_catruns,n_test,data_image_size,data_image_size)
      y_train = jnp.array(y_train).reshape(number_different_catruns,int((n_samples/batch_size)),batch_size)
      y_test = jnp.array(y_test).reshape(number_different_catruns,n_test)
      
      opt_state = opt_init(MLP_params)
      params_with_adam = get_params(opt_state)


      #print("Shape x_train",np.shape(x_train))
      #print("Shape y_train",np.shape(y_train))
      #print("Shape x_test",np.shape(x_test))
      #print("Shape y_test",np.shape(y_test))
      
      #print("\tLänge Offspring List:",len(offspring_list))
      #print("Overhead data time:", time.time() - start_data_collection_time)
      for i in range(len(offspring_list)):
        
        conv_weights = offspring_list[i]
        
        result_off,params_trained = jit_vmap_bootstrapp_offspring_MLP(params_with_adam,opt_state,conv_weights,x_train,y_train,x_test,y_test)
        result_list_metaepoch.append(result_off)
      results_overall.append(result_list_metaepoch)


    '''Assess Metaepoch'''
    '''means over all cat combis for every offspring and oepoch'''
    means_of_every_offspring = np.mean(results_overall,axis = 2)
    '''average standard deviation of the mean of every offspring, over all oepochs. How accurate is the fitness evaluation of one offspring?'''
    avg_std = np.mean(np.var(means_of_every_offspring, axis = 0))**0.5


    logg(f"\taverage standard deviation of the mean of every offspring: {avg_std:.4f}", ) 
    results_for_every_offspring = np.mean(np.mean(results_overall, axis = 0),axis = 1) #stimmt, checked 29.08
    logg("\tMetaepoch mean: {:.4f}, std: {:.4f}".format(np.mean(results_overall),np.std(results_for_every_offspring))) #stimmt, checked 29.08
    logg("\tMetaepoch max performer: {:.4f}, min performer: {:.4f}".format(np.max(results_for_every_offspring),np.min(results_for_every_offspring)))
    logg("\tTime per metaepoch:{:.1f}s\n".format(time.time() - start_meta))

    if any(item in randomizer_list for item in np.argpartition(results_for_every_offspring, -5)[-5:]):
      print("\t+++Top 5 results include randomizer offspring+++")

    '''Check for best performer'''
    metaepoch_superhero = None
    for idx, acc_value in enumerate(results_for_every_offspring):
        if acc_value>best_performer:
          best_performer = acc_value
          metaepoch_superhero = idx
    if metaepoch_superhero:
      if metaepoch_superhero in randomizer_list:
        print("+++Best weight delivered by randomizer+++")
      best_performer = results_for_every_offspring[metaepoch_superhero]
      best_weights = offspring_list[metaepoch_superhero]
      with open(save_path+f"best_weight_{best_performer:.4f}.pkl", 'wb') as f:
        pickle.dump(best_weights, f, pickle.HIGHEST_PROTOCOL)
        f.close()
      logg(f"New best performer mean: {best_performer:.4f}")

      '''FIFO for best weight queue, only one item per metaepoch is added to increase diversity'''
      if use_best_weight_queue:
        best_weight_queue.append(offspring_list[metaepoch_superhero])
        if len(best_weight_queue)> len_best_weight_queue:
          best_weight_queue = best_weight_queue[-len_best_weight_queue:]


    if np.mean(results_overall) > best_father_mean:
      best_father_mean = np.mean(results_overall)
      best_father_queue.append(father_weights)
      if len(best_father_queue)> len_best_father_queue:
        best_father_queue = best_father_queue[-len_best_father_queue:]


on google
Log path: /content/drive/MyDrive/Colab Notebooks/18.10.2022_18.56/
n_training_epochs = 10
n_offsp_epoch = 5
n_samples = 25
n_test = 75
batch_size = 5
std_modifier = 0.01
use_sigma_decay = True
sigma_start = 0.05
sigma_goal = 0.01
n_decay_epochs = 500
use_pickle = False
pickle_path = ./pretrained_params/best_weight_0.8463.pkl
NNin1 = 50
NNout1 = 5
Convu_in1 = 32
Convu2_in = 64
GPU: Tesla T4
	load known data for metaepoch:0
	average standard deviation of the mean of every offspring: 0.0196
	Metaepoch mean: 0.4787, std: 0.0173
	Metaepoch max performer: 0.5073, min performer: 0.4028
	Time per metaepoch:76.0s

New best performer mean: 0.5073
	load known data for metaepoch:1


KeyboardInterrupt: ignored